# Experimentation
This notebook is used for building and experimenting using parcel data retrieved from the Regrid API.

## Import Required Libraries

In [2]:
# Import Required Libraries
from app import main
import pandas as pd
import numpy as np
import os
import json
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load and Explore Data
Use the regrid API connector in main.py to retrieve parcel data and load it into a pandas DataFrame. Perform basic exploration of the data.

In [3]:
# Load and Explore Data
# Retrieve parcel data using the regrid API connector
API = os.getenv('REGRID_API_KEY')
connector = main.RegridAPIConnector(API)
parcel_data = connector.get_parcel_by_address(address='8522 Sweetwood Dr', path='/us/tx/dallas')


print(parcel_data)

{'parcels': {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-96.697886, 32.8049195], [-96.6979145, 32.8049195], [-96.697943, 32.8049225], [-96.698012, 32.804595], [-96.697866, 32.8044685], [-96.697621, 32.8046735], [-96.697805, 32.804941], [-96.6978305, 32.8049305], [-96.697858, 32.8049235], [-96.697886, 32.8049195]]]}, 'properties': {'headline': '8522 Sweetwood Dr', 'path': '/us/tx/dallas/northeast-dallas/717023', 'fields': {'ogc_fid': 717023, 'geoid': '48113', 'parcelnumb': '00000667798000000', 'parcelnumb_no_formatting': '00000667798000000', 'account_number': '00000667798000000', 'usecode': '1', 'usedesc': 'SINGLE FAMILY RESIDENCES', 'zoning': 'R-7.5(A', 'zoning_description': 'Single Family District 7500 Square Feet', 'yearbuilt': 1961, 'structstyle': '15', 'parvaltype': 'ASSESSED', 'improvval': 355670.0, 'landval': 115000.0, 'parval': 470670.0, 'owner': 'RICHARDSON LAWRENCE G', 'owner2': 'VICTORIA R', 'mailadd': '8522

In [4]:
# Display parcel_data JSON in an HTML wrapper
from IPython.display import display, HTML
import json
 
# Convert JSON to a formatted HTML string
html_content = f'<pre>{json.dumps(parcel_data, indent=4)}</pre>'
display(HTML(html_content))

In [5]:
# Export parcel_data as a JSON object
with open('parcel_data.json', 'w') as json_file:
    json.dump(parcel_data, json_file, indent=4)
print("parcel_data has been exported to parcel_data.json")

parcel_data has been exported to parcel_data.json
